#  Продажи продуктов через телемаркетинг

## Каждый файл содержит, как минимум, 4 колонки (поля):
* FILIAL_ID
* SUBS_ID
* PROD_ID
* ACT_DTTM

## Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе

## Особенности данных:

1. Сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
2. Поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
3. Продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
4. Если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
5. Сохранить результат в датафрэйм с разделителем ';', содержащий корректные подключения

In [1]:
# Импортируем библиотеки

import pandas as pd
import os
from pathlib import Path

In [2]:
# Указываем путь до файла

path_to_file = 'subsid'

In [3]:
 # Смотрим, какие файлы лежат в директории

os.listdir(path_to_file)

['tm_sales_1.csv',
 'tm_sales_3.csv',
 'tm_sales_2.csv',
 'prod_activations_logs.csv']

In [4]:
# Помещаем пути к файлам в генератор tms

tms = Path(path_to_file).glob('tm*.csv')

In [5]:
# Объединяем прочитанные csv файлы в один

df = pd.DataFrame()

for tm in tms:
    temp_df = pd.read_csv(tm, sep=';')
    
    df = pd.concat((df, temp_df))

In [6]:
df.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21


In [7]:
# Избавляемся от NaN значений в колонке SUBS_ID

df = df.dropna(subset=['SUBS_ID'])

In [8]:
# Перед некоторыми id нет букв 'id', а только номер
# Напишем функцию, которая исправляет это

def id_adder(value):
    if value.startswith('id'):
        return value
    return 'id' + value

In [9]:
# Применим функцию к столбцу SUBS_ID

df.SUBS_ID = df.SUBS_ID.apply(id_adder)
df = df.reset_index()
df.SUBS_ID

0    id4651830
1    id7646509
2    id7412683
3    id5764122
4    id7642700
5    id1374509
6    id5416547
7    id8362218
8    id2185490
Name: SUBS_ID, dtype: object

In [10]:
# Считаем файл с логами

logs = list(Path(path_to_file).glob('*logs*'))[0]
logs = pd.read_csv(logs, sep=';')
logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43


In [11]:
# Объединим два датафрейма

full_data = df.merge(logs, on=['SUBS_ID', 'PROD_ID'])
full_data.head()

,index,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,0,id4651830,1,1954,20-03-2020 14:59,20-03-2020 14:59,01-12-2020 00:00
1,1,id7646509,5,6431,19-03-2020 13:00,19-03-2020 13:00,19-03-2020 13:03
2,1,id7642700,6,3020,15-03-2020 14:21,15-03-2020 14:21,15-03-2020 23:42
3,0,id5416547,3,1743,17-03-2020 10:17,17-03-2020 10:17,25-03-2020 11:00
4,3,id2185490,2,3210,16-03-2020 16:28,16-03-2020 16:28,01-12-2020 00:00


In [12]:
# Переведем колонки ACT_DTTM, START_DTTM и END_DTTM в формат даты

full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] = full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] \
                                                    .apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y %H:%M'))
# или 
# full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] = full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] \
#                                                   .apply(pd.to_datetime, format='%d-%m-%Y %H:%M')

full_data.head()

,index,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00
1,1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,1,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00
3,0,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00
4,3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00


In [13]:
# Найдем временную разность между END_DTTM и START_DTTM

full_data['difference'] = full_data.END_DTTM - full_data.START_DTTM

In [14]:
full_data.head()

,index,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,1,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
3,0,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00


In [17]:
# Посмотрим засчитанные продажи

greater_than_5_min = full_data.difference > '5m'

In [18]:
df = full_data.loc[greater_than_5_min]

In [19]:
df[['SUBS_ID', 'PROD_ID', 'difference']].to_csv('my_tm.csv', sep=';', index=False)